In [2]:
# ============================================================
# 0. Setup – Imports, display, and paths
# ============================================================
import os
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 140)

# Paths (adapt if your structure is slightly different)
NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = NOTEBOOK_DIR.parent  # assumes notebook in /notebooks

DATA_DIR = PROJECT_ROOT / "data"
DATA_RAW = DATA_DIR / "raw"
DATA_PROCESSED = DATA_DIR / "processed"
DATA_EXTERNAL = DATA_DIR / "external"  # optional: external sources (e.g. OPDI, weather)

print("Notebook directory:", NOTEBOOK_DIR)
print("Project root:", PROJECT_ROOT)
print("Raw data path:", DATA_RAW)
print("Processed data path:", DATA_PROCESSED)
print("External data path:", DATA_EXTERNAL)


Notebook directory: c:\Users\lenovo\Desktop\flight-delay-prediction-and-operational-efficiency\notebooks
Project root: c:\Users\lenovo\Desktop\flight-delay-prediction-and-operational-efficiency
Raw data path: c:\Users\lenovo\Desktop\flight-delay-prediction-and-operational-efficiency\data\raw
Processed data path: c:\Users\lenovo\Desktop\flight-delay-prediction-and-operational-efficiency\data\processed
External data path: c:\Users\lenovo\Desktop\flight-delay-prediction-and-operational-efficiency\data\external


In [3]:
# ============================================================
# 1. Helper Functions
# ============================================================

def clean_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Standardize column names:
    - strip spaces
    - lowercase
    - replace spaces with underscores
    - remove non-alphanumeric characters (except underscore)
    """
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_")
        .str.replace(r"[^a-zA-Z0-9_]", "", regex=True)
    )
    return df


def quick_info(df: pd.DataFrame, name: str = ""):
    """Print basic info for a DataFrame."""
    print(f"\n=== {name} ===")
    print("Shape:", df.shape)
    print("Columns:", df.columns.tolist())
    print("\nDtypes:")
    print(df.dtypes)
    print("\nMissing values (top 10):")
    print(df.isna().sum().sort_values(ascending=False).head(10))


def find_delay_columns(df: pd.DataFrame):
    """
    Heuristic: find columns that likely refer to delay.
    This avoids hardcoding names in case the dataset uses different labels.
    """
    return [c for c in df.columns if "delay" in c.lower()]


def find_cancel_columns(df: pd.DataFrame):
    """
    Heuristic: find columns that likely refer to cancellation.
    """
    return [c for c in df.columns if "cancel" in c.lower() or "cncl" in c.lower()]


## 📂 2. Load Raw Datasets

In this section, we load all raw input datasets as referenced in the project structure:

- `10866_2024.csv`, `10866_2025.csv` → core flight-level records (including schedule & delay)  
- `airport_traffic_2024.csv`, `airport_traffic_2025.csv` → aggregated airport traffic  
- `airports.csv`, `runways.csv` → airport & runway attributes  
- `flights_bavaria_2025.csv` → regional subset (Bavaria flights)  
- `Stakeholder_Punctuality.xlsx` → stakeholder-oriented KPI targets / reference punctuality

We also optionally load any **external** sources (e.g., OPDI Germany data) from `data/external/`.


In [4]:
# ============================================================
# 2. Load Raw Datasets
# ============================================================

flights_2024 = pd.read_csv(DATA_RAW / "10866_2024.csv")
flights_2025 = pd.read_csv(DATA_RAW / "10866_2025.csv")

traffic_2024 = pd.read_csv(DATA_RAW / "airport_traffic_2024.csv")
traffic_2025 = pd.read_csv(DATA_RAW / "airport_traffic_2025.csv")

airports = pd.read_csv(DATA_RAW / "airports.csv")
runways = pd.read_csv(DATA_RAW / "runways.csv")

bavaria_2025 = pd.read_csv(DATA_RAW / "flights_bavaria_2025.csv")

kpi_excel = pd.read_excel(DATA_RAW / "Stakeholder_Punctuality.xlsx")

print("Raw datasets loaded:")
for name, df in {
    "flights_2024": flights_2024,
    "flights_2025": flights_2025,
    "traffic_2024": traffic_2024,
    "traffic_2025": traffic_2025,
    "airports": airports,
    "runways": runways,
    "bavaria_2025": bavaria_2025,
    "kpi_excel": kpi_excel,
}.items():
    print(f"  {name:15s} → {df.shape}")


Raw datasets loaded:
  flights_2024    → (8784, 24)
  flights_2025    → (7839, 24)
  traffic_2024    → (115486, 13)
  traffic_2025    → (96635, 13)
  airports        → (83798, 19)
  runways         → (47215, 20)
  bavaria_2025    → (103174, 17)
  kpi_excel       → (6, 5)


In [5]:
# ============================================================
# 2.b Optional – Load External Sources (if present)
# ============================================================

external_files = {}
if (DATA_EXTERNAL / "germany_flights_2022.csv").exists():
    external_files["germany_2022"] = pd.read_csv(DATA_EXTERNAL / "germany_flights_2022.csv")
if (DATA_EXTERNAL / "germany_flights_2023.csv").exists():
    external_files["germany_2023"] = pd.read_csv(DATA_EXTERNAL / "germany_flights_2023.csv")
if (DATA_EXTERNAL / "germany_flights_2024.csv").exists():
    external_files["germany_2024"] = pd.read_csv(DATA_EXTERNAL / "germany_flights_2024.csv")

for name, df in external_files.items():
    print(f"{name:15s} → {df.shape}")


C:\Users\lenovo\AppData\Local\Temp\ipykernel_8356\2189495507.py:7: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  external_files["germany_2022"] = pd.read_csv(DATA_EXTERNAL / "germany_flights_2022.csv")


germany_2022    → (1671203, 17)
germany_2023    → (1737121, 17)
germany_2024    → (1085301, 17)


In [6]:
# ============================================================
# 3. Clean Column Names
# ============================================================

flights_2024 = clean_columns(flights_2024)
flights_2025 = clean_columns(flights_2025)
traffic_2024 = clean_columns(traffic_2024)
traffic_2025 = clean_columns(traffic_2025)
airports = clean_columns(airports)
runways = clean_columns(runways)
bavaria_2025 = clean_columns(bavaria_2025)
kpi_excel = clean_columns(kpi_excel)

for name, df in external_files.items():
    external_files[name] = clean_columns(df)

print("Columns cleaned. Example – flights_2024 columns:")
print(flights_2024.columns.tolist())


Columns cleaned. Example – flights_2024 columns:
['year', 'month', 'day', 'hour', 'temp', 'temp_source', 'rhum', 'rhum_source', 'prcp', 'prcp_source', 'wdir', 'wdir_source', 'wspd', 'wspd_source', 'wpgt', 'wpgt_source', 'pres', 'pres_source', 'tsun', 'tsun_source', 'cldc', 'cldc_source', 'coco', 'coco_source']


## 📊 3. Dataset Inventory & Basic Checks

Before diving into delays and KPIs, we create a clear overview:

- Shapes and schemas of all key tables  
- High-level missingness  
- Identification of delay-related columns in the main flights datasets  

This mirrors how an analytics team would validate data quality at the start of a real project.


In [7]:
# ============================================================
# 3.1 – Dataset Inventory Overview
# ============================================================

datasets = {
    "flights_2024": flights_2024,
    "flights_2025": flights_2025,
    "traffic_2024": traffic_2024,
    "traffic_2025": traffic_2025,
    "airports": airports,
    "runways": runways,
    "bavaria_2025": bavaria_2025,
    "kpi_excel": kpi_excel,
}

for name, df in datasets.items():
    print(f"{name:15s} → shape: {df.shape}")


flights_2024    → shape: (8784, 24)
flights_2025    → shape: (7839, 24)
traffic_2024    → shape: (115486, 13)
traffic_2025    → shape: (96635, 13)
airports        → shape: (83798, 19)
runways         → shape: (47215, 20)
bavaria_2025    → shape: (103174, 17)
kpi_excel       → shape: (6, 5)


In [8]:
# ============================================================
# 3.2 – Quick Info: Core Flight Datasets
# ============================================================

quick_info(flights_2024, "flights_2024")
quick_info(flights_2025, "flights_2025")



=== flights_2024 ===
Shape: (8784, 24)
Columns: ['year', 'month', 'day', 'hour', 'temp', 'temp_source', 'rhum', 'rhum_source', 'prcp', 'prcp_source', 'wdir', 'wdir_source', 'wspd', 'wspd_source', 'wpgt', 'wpgt_source', 'pres', 'pres_source', 'tsun', 'tsun_source', 'cldc', 'cldc_source', 'coco', 'coco_source']

Dtypes:
year             int64
month            int64
day              int64
hour             int64
temp           float64
temp_source     object
rhum             int64
rhum_source     object
prcp           float64
prcp_source     object
wdir             int64
wdir_source     object
wspd           float64
wspd_source     object
wpgt           float64
wpgt_source    float64
pres           float64
pres_source     object
tsun           float64
tsun_source     object
cldc           float64
cldc_source     object
coco           float64
coco_source     object
dtype: object

Missing values (top 10):
wpgt_source    8784
wpgt           8784
tsun_source    2267
tsun           2267
coco   

## ⏱️ 4. Flight Delays – Columns & Target Definition

The README defines the **delay prediction target** as:

> `delay_15 = 1 if delay >= 15 minutes else 0`

In this section, we:

1. Identify which columns correspond to **arrival / departure delay**  
2. Inspect their distributions and missing values  
3. Prepare the boolean target `delay_15` (exploratory version) for 2024  
4. Compute basic punctuality KPIs:
   - on-time rate
   - average delay
   - cancellation rate (if available)


In [9]:
# ============================================================
# 4.1 – Identify Delay & Cancellation Columns (Heuristic)
# ============================================================

delay_cols_2024 = find_delay_columns(flights_2024)
cancel_cols_2024 = find_cancel_columns(flights_2024)

print("Delay-related columns in flights_2024:", delay_cols_2024)
print("Cancellation-related columns in flights_2024:", cancel_cols_2024)


Delay-related columns in flights_2024: []
Cancellation-related columns in flights_2024: []


In [10]:
# ============================================================
# 4.2 – Basic Delay Distribution (Flights 2024)
# ============================================================

# TODO: adjust this to your actual main delay column name
# For example: main_delay_col = "arrival_delay" or "arr_delay"
# Here we try to pick the first delay column as a default
main_delay_col = delay_cols_2024[0] if delay_cols_2024 else None
print("Using main delay column:", main_delay_col)

if main_delay_col is not None:
    flights_2024[main_delay_col] = pd.to_numeric(
        flights_2024[main_delay_col], errors="coerce"
    )
    print(flights_2024[main_delay_col].describe())
else:
    print("No delay column detected automatically – please set main_delay_col manually.")


Using main delay column: None
No delay column detected automatically – please set main_delay_col manually.


In [11]:
# ============================================================
# 4.3 – Create delay_15 Target & Punctuality KPIs (2024)
# ============================================================

DELAY_THRESHOLD = 15  # minutes

if main_delay_col is not None:
    flights_2024["delay_15"] = (flights_2024[main_delay_col] >= DELAY_THRESHOLD).astype("Int64")

    on_time_rate = 1 - flights_2024["delay_15"].mean()
    avg_delay = flights_2024[main_delay_col].mean()

    print(f"On-time rate (delay < {DELAY_THRESHOLD} min): {on_time_rate:.2%}")
    print(f"Average delay (all flights): {avg_delay:.2f} minutes")

    if cancel_cols_2024:
        # Again, pick the first cancellation column as default
        main_cancel_col = cancel_cols_2024[0]
        flights_2024[main_cancel_col] = flights_2024[main_cancel_col].astype("Int64")
        cancel_rate = flights_2024[main_cancel_col].mean()
        print(f"Cancellation rate (2024): {cancel_rate:.2%}")
    else:
        print("No cancellation column detected – skipping cancellation KPIs.")
else:
    print("Please define main_delay_col before computing delay_15.")


Please define main_delay_col before computing delay_15.


## 🧭 5. Airport, Traffic, and Runway Context

To understand delays in context, we inspect:

- Airport metadata (`airports.csv`)  
- Runway information (`runways.csv`)  
- Airport traffic summaries (`airport_traffic_2024/2025.csv`)  

This helps later when we:

- engineer airport capacity features  
- relate delay levels to traffic load  
- analyze differences between airports.


In [12]:
# ============================================================
# 5.1 – Airports & Traffic Overview
# ============================================================

quick_info(airports, "airports")
quick_info(runways, "runways")
quick_info(traffic_2024, "traffic_2024")
quick_info(traffic_2025, "traffic_2025")



=== airports ===
Shape: (83798, 19)
Columns: ['id', 'ident', 'type', 'name', 'latitude_deg', 'longitude_deg', 'elevation_ft', 'continent', 'iso_country', 'iso_region', 'municipality', 'scheduled_service', 'icao_code', 'iata_code', 'gps_code', 'local_code', 'home_link', 'wikipedia_link', 'keywords']

Dtypes:
id                     int64
ident                 object
type                  object
name                  object
latitude_deg         float64
longitude_deg        float64
elevation_ft         float64
continent             object
iso_country           object
iso_region            object
municipality          object
scheduled_service     object
icao_code             object
iata_code             object
gps_code              object
local_code            object
home_link             object
wikipedia_link        object
keywords              object
dtype: object

Missing values (top 10):
home_link         79401
iata_code         74730
icao_code         74515
wikipedia_link    67216
key

In [13]:
# ============================================================
# 5.2 – Top Airports by Traffic (2024)
# ============================================================

# TODO: adjust column names if needed (e.g. 'airport', 'flights', 'movements')
print("traffic_2024 columns:", traffic_2024.columns.tolist())

# Guess an airport id column and a traffic metric:
# For example: 'airport_icao' and 'total_movements'
# You will need to adapt this based on your actual columns.
# Example generic pattern:

# Try to guess an airport column
airport_col_candidates = [c for c in traffic_2024.columns if "airport" in c.lower() or "icao" in c.lower()]
traffic_col_candidates = [c for c in traffic_2024.columns if "mov" in c.lower() or "traffic" in c.lower() or "flights" in c.lower()]

print("Guessed airport columns:", airport_col_candidates)
print("Guessed traffic metric columns:", traffic_col_candidates)


traffic_2024 columns: ['year', 'month_num', 'month_mon', 'flt_date', 'apt_icao', 'apt_name', 'state_name', 'flt_dep_1', 'flt_arr_1', 'flt_tot_1', 'flt_dep_ifr_2', 'flt_arr_ifr_2', 'flt_tot_ifr_2']
Guessed airport columns: ['apt_icao']
Guessed traffic metric columns: []


## 🌍 6. Optional – External Context (Germany-wide OPDI Flights)

If external Germany flight datasets (e.g., OPDI-based extractions) are available in `data/external/`,  
we can briefly explore them here to see how **2024–2025 punctuality patterns** sit inside the broader German traffic context.

This section is optional and can be expanded into a dedicated notebook if desired.


In [15]:
# ============================================================
# 6.1 – External Germany Data (if present)
# ============================================================

if external_files:
    for name, df in external_files.items():
        quick_info(df, name)
else:
    print("No external Germany datasets found in data/external/.")



=== germany_2022 ===
Shape: (1671203, 17)
Columns: ['id', 'icao24', 'flt_id', 'dof', 'adep', 'ades', 'adep_p', 'ades_p', 'registration', 'model', 'typecode', 'icao_aircraft_class', 'icao_operator', 'first_seen', 'last_seen', 'version', 'unix_time']

Dtypes:
id                     object
icao24                 object
flt_id                 object
dof                    object
adep                   object
ades                   object
adep_p                 object
ades_p                 object
registration           object
model                  object
typecode               object
icao_aircraft_class    object
icao_operator          object
first_seen             object
last_seen              object
version                object
unix_time               int64
dtype: object

Missing values (top 10):
ades_p                 1627555
adep_p                 1621163
icao_operator           636670
model                   429528
adep                    339571
ades                    283814
icao_

## 💾 7. Export Cleaned Subsets (for Next Steps)

To keep a clean pipeline between notebooks, we export:

- A cleaned 2024 flight dataset with delay & target column (`delay_15`)  
- Optionally, a slimmed-down version with only essential modeling fields  
- These will be used in:

  - `02_feature_engineering.ipynb`  
  - `03_model_training.ipynb`  
  - `04_dashboard_prep.ipynb`


In [16]:
# ============================================================
# 7.1 – Export Cleaned 2024 Flights Dataset
# ============================================================

DATA_PROCESSED.mkdir(parents=True, exist_ok=True)

if main_delay_col is not None and "delay_15" in flights_2024.columns:
    out_path_full = DATA_PROCESSED / "flights_2024_clean_full.csv"
    flights_2024.to_csv(out_path_full, index=False)
    print("Saved cleaned full 2024 flights to:", out_path_full)

    # Example of a slim version for modeling (adjust feature list later)
    core_cols = [c for c in flights_2024.columns if "delay" in c or "time" in c or "date" in c or "airport" in c] + ["delay_15"]
    core_cols = list(dict.fromkeys(core_cols))  # deduplicate
    flights_2024_core = flights_2024[core_cols].copy()

    out_path_core = DATA_PROCESSED / "flights_2024_core_model.csv"
    flights_2024_core.to_csv(out_path_core, index=False)
    print("Saved core 2024 modeling dataset to:", out_path_core)
else:
    print("delay_15 or delay column missing – skipping export. Define main_delay_col and rerun section 4.")


delay_15 or delay column missing – skipping export. Define main_delay_col and rerun section 4.
